# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 13 2022 1:27PM,241786,21,8SAMPLE-403040419,"NBTY Global, Inc.",Released
1,Jul 13 2022 1:26PM,241785,20,ELSE-8269541,Else Nutrition,Released
2,Jul 13 2022 1:23PM,241784,21,25732-02,"NBTY Global, Inc.",Released
3,Jul 13 2022 1:13PM,241782,20,ALUR198168851,Alumier Labs Inc.,Released
4,Jul 13 2022 1:13PM,241782,20,ALUR383595124,Alumier Labs Inc.,Released
5,Jul 13 2022 1:13PM,241782,20,ALUR498215764,Alumier Labs Inc.,Released
6,Jul 13 2022 1:13PM,241782,20,ALUR520035521,Alumier Labs Inc.,Released
7,Jul 13 2022 1:13PM,241782,20,ALUR897982225,Alumier Labs Inc.,Released
8,Jul 13 2022 1:13PM,241782,20,ALUR952576410,Alumier Labs Inc.,Released
9,Jul 13 2022 1:13PM,241782,20,ALUR615204537,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,241781,Executing,3
37,241782,Released,14
38,241784,Released,1
39,241785,Released,1
40,241786,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241781,NaN,3.0,NaN
241782,NaN,NaN,14.0
241784,NaN,NaN,1.0
241785,NaN,NaN,1.0
241786,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241679,0.0,1.0,0.0
241705,0.0,0.0,1.0
241706,0.0,1.0,0.0
241710,9.0,16.0,1.0
241711,0.0,13.0,16.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241679,0,1,0
241705,0,0,1
241706,0,1,0
241710,9,16,1
241711,0,13,16


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241679,0,1,0
1,241705,0,0,1
2,241706,0,1,0
3,241710,9,16,1
4,241711,0,13,16


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241679,,1,
1,241705,,,1
2,241706,,1,
3,241710,9,16,1
4,241711,,13,16


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc."
1,Jul 13 2022 1:26PM,241785,20,Else Nutrition
2,Jul 13 2022 1:23PM,241784,21,"NBTY Global, Inc."
3,Jul 13 2022 1:13PM,241782,20,Alumier Labs Inc.
17,Jul 13 2022 1:12PM,241781,19,"AdvaGen Pharma, Ltd"
20,Jul 13 2022 1:05PM,241779,19,"GCH Granules USA, Inc."
21,Jul 13 2022 1:03PM,241777,20,Alumier Labs Inc.
22,Jul 13 2022 1:03PM,241775,10,"Methapharm, Inc."
38,Jul 13 2022 1:03PM,241778,10,"Citieffe, Inc."
39,Jul 13 2022 1:01PM,241776,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc.",,,1
1,Jul 13 2022 1:26PM,241785,20,Else Nutrition,,,1
2,Jul 13 2022 1:23PM,241784,21,"NBTY Global, Inc.",,,1
3,Jul 13 2022 1:13PM,241782,20,Alumier Labs Inc.,,,14
4,Jul 13 2022 1:12PM,241781,19,"AdvaGen Pharma, Ltd",,3,
5,Jul 13 2022 1:05PM,241779,19,"GCH Granules USA, Inc.",,1,
6,Jul 13 2022 1:03PM,241777,20,Alumier Labs Inc.,,,1
7,Jul 13 2022 1:03PM,241775,10,"Methapharm, Inc.",,,16
8,Jul 13 2022 1:03PM,241778,10,"Citieffe, Inc.",,,1
9,Jul 13 2022 1:01PM,241776,10,"Methapharm, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc.",1,,
1,Jul 13 2022 1:26PM,241785,20,Else Nutrition,1,,
2,Jul 13 2022 1:23PM,241784,21,"NBTY Global, Inc.",1,,
3,Jul 13 2022 1:13PM,241782,20,Alumier Labs Inc.,14,,
4,Jul 13 2022 1:12PM,241781,19,"AdvaGen Pharma, Ltd",,3,
5,Jul 13 2022 1:05PM,241779,19,"GCH Granules USA, Inc.",,1,
6,Jul 13 2022 1:03PM,241777,20,Alumier Labs Inc.,1,,
7,Jul 13 2022 1:03PM,241775,10,"Methapharm, Inc.",16,,
8,Jul 13 2022 1:03PM,241778,10,"Citieffe, Inc.",1,,
9,Jul 13 2022 1:01PM,241776,10,"Methapharm, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc.",1,,
1,Jul 13 2022 1:26PM,241785,20,Else Nutrition,1,,
2,Jul 13 2022 1:23PM,241784,21,"NBTY Global, Inc.",1,,
3,Jul 13 2022 1:13PM,241782,20,Alumier Labs Inc.,14,,
4,Jul 13 2022 1:12PM,241781,19,"AdvaGen Pharma, Ltd",,3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jul 13 2022 1:26PM,241785,20,Else Nutrition,1.0,NaN,NaN
2,Jul 13 2022 1:23PM,241784,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jul 13 2022 1:13PM,241782,20,Alumier Labs Inc.,14.0,NaN,NaN
4,Jul 13 2022 1:12PM,241781,19,"AdvaGen Pharma, Ltd",NaN,3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 13 2022 1:27PM,241786,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jul 13 2022 1:26PM,241785,20,Else Nutrition,1.0,0.0,0.0
2,Jul 13 2022 1:23PM,241784,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jul 13 2022 1:13PM,241782,20,Alumier Labs Inc.,14.0,0.0,0.0
4,Jul 13 2022 1:12PM,241781,19,"AdvaGen Pharma, Ltd",0.0,3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2659295,73.0,3.0,0.0
12,241771,1.0,0.0,0.0
15,725133,23.0,29.0,9.0
16,1208689,1.0,24.0,0.0
19,725300,1.0,4.0,0.0
20,1692259,30.0,19.0,0.0
21,966981,3.0,1.0,0.0
22,483493,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2659295,73.0,3.0,0.0
1,12,241771,1.0,0.0,0.0
2,15,725133,23.0,29.0,9.0
3,16,1208689,1.0,24.0,0.0
4,19,725300,1.0,4.0,0.0
5,20,1692259,30.0,19.0,0.0
6,21,966981,3.0,1.0,0.0
7,22,483493,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,73.0,3.0,0.0
1,12,1.0,0.0,0.0
2,15,23.0,29.0,9.0
3,16,1.0,24.0,0.0
4,19,1.0,4.0,0.0
5,20,30.0,19.0,0.0
6,21,3.0,1.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,73.0
1,12,Released,1.0
2,15,Released,23.0
3,16,Released,1.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0
Executing,3.0,0.0,29.0,24.0,4.0,19.0,1.0,0.0
Released,73.0,1.0,23.0,1.0,1.0,30.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,0.0,29.0,24.0,4.0,19.0,1.0,0.0
2,Released,73.0,1.0,23.0,1.0,1.0,30.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,0.0,29.0,24.0,4.0,19.0,1.0,0.0
2,Released,73.0,1.0,23.0,1.0,1.0,30.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()